<a href="https://colab.research.google.com/github/batmanvane/complex-systems-modeling/blob/main/notebooks/chaos_game_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Complex Systems Modeling — Session 2 **
### From Coordinates & Randomness → Midpoint Rule → Chaos Game → Sierpiński Triangle & Carpet

Welcome! This notebook is a **self‑explanatory tutorial**. Each section introduces a concept, gives you a small **task**, and then provides a **commented solution** you can reveal by uncommenting lines.

**What you'll build (step‑by‑step):**
1. Plot points in a simple coordinate system
2. Use randomness to move points
3. Apply the **midpoint rule** (“halving toward a target”)
4. Combine ideas to make the **Sierpiński Triangle** (Chaos Game)
5. Extend to the **Sierpiński Carpet** (mask + Chaos Game/IFS)

> Tips: Run cells top to bottom. When you see ** Solution**, read it first, then uncomment to try it.



## Table of Contents
1. [Setup](#setup)  
2. [Coordinates & Plotting](#coords)  
3. [Randomness: Selecting Points](#random)  
4. [Midpoint (Halving) Rule](#midpoint)  
5. [Project 1: Sierpiński Triangle (Chaos Game)](#triangle)  
6. [Project 2: Sierpiński Carpet](#carpet)  
7. [Explore & Reflect](#reflect)



---
<a id="setup"></a>
## 1) Setup


In [ ]:

# Imports
import numpy as np
import matplotlib.pyplot as plt
import random
import time

def scatter_points(points, title="Points", figsize=(6,6), s=8):
    fig, ax = plt.subplots(figsize=figsize)
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    ax.scatter(xs, ys, s=s, alpha=0.9)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.25)
    ax.set_title(title)
    plt.show()

print(" Setup complete.")



---
<a id="coords"></a>
## 2) Coordinates & Plotting

We’ll start with a few points in the unit square and plot them.



### Task 2.1 — Create and plot 5 points inside the unit square
- Build a list of 5 `(x, y)` pairs with values in `[0, 1]`.
- Use `scatter_points(...)` to visualize them.


In [ ]:

# TODO: Define five points in [0,1]x[0,1] use datastructures tuples and lists


# Solution (uncomment):
# points = [(0.2,0.3),(0.8,0.4),(0.6,0.9),(0.1,0.8),(0.4,0.1)]

# Visualize
# scatter_points(points, title="Five points in the unit square")



---
<a id="random"></a>
## 3) Randomness: Selecting Points

We’ll sample random points and practice basic plotting.



### Task 3.1 — Generate 200 random points in the unit square
- Use `random.random()` or `np.random.rand()`.
- Plot them.


In [ ]:

# TODO: Generate random points

rand_points = [] #initialize an empty list


# Solution (uncomment):
# rand_points = [(random.random(), random.random()) for _ in range(200)]

# Visualize
# scatter_points(rand_points, title="Random points in [0,1]²", s=6)



---
<a id="midpoint"></a>
## 4) Midpoint (Halving) Rule

Given a current position `C` and a target `T`, the **midpoint** is `M = C + 0.5*(T - C)`.  
This simple rule will power the **Chaos Game**.



### Task 4.1 — Implement a `midpoint(C, T, fraction=0.5)` function
- Return the point that moves a fraction (default 0.5) of the way from `C` to `T`.
- Test it with `C=(0.9,0.9)` and `T=(0.0,0.0)`.


In [ ]:
#  Solution (uncomment):
# def midpoint(C, T, fraction=0.5):
#     """
#      meaningfull docstring
#      """
#     cx, cy = C
#     tx, ty = T
#     nx = cx + fraction * (tx - cx)
#     ny = cy + fraction * (ty - cy)
#     return (nx, ny)

# # Quick test
# print("Midpoint test:", midpoint((0.9,0.9),(0.0,0.0)))



---
<a id="triangle"></a>
## 5) Project 1: Sierpiński Triangle (Chaos Game)

**Idea:** Choose one of three triangle vertices at random and move halfway from the current point toward that vertex. Repeat!

We’ll build this progressively.



### 5.1 Define triangle vertices
Use an equilateral triangle in the unit square.


In [ ]:

# Equilateral triangle vertices
triangle_vertices = [(0.0, 0.0), (1.0, 0.0), (0.5, np.sqrt(3)/2)]
triangle_vertices



### Task 5.2 — Implement a single Chaos-Game step
- Randomly choose one of the three vertices
- Move the current point halfway toward that vertex using `midpoint`.


In [ ]:
# Solution (uncomment):
# def chaos_game_step(current_point, vertices, fraction=0.5):
#     chosen = random.choice(vertices)
#     return midpoint(current_point, chosen, fraction), chosen

# # Quick test (after you implemented midpoint):
# # print(chaos_game_step((0.5,0.5), triangle_vertices))



### Task 5.3 — Run the Chaos Game
- Start at any point (e.g., the centroid).
- Iterate N times, skipping the first ~100 transient points.
- Plot the result.


In [ ]:
# Solution (uncomment):
# def play_chaos_game(vertices, start_point, num_steps=15000, fraction=0.5, skip_initial=100):
#     points = [start_point]
#     chosen_vertices = []
#     cur = start_point
#     for _ in range(num_steps):
#         cur, chosen = chaos_game_step(cur, vertices, fraction)
#         points.append(cur); chosen_vertices.append(chosen)
#     return points[skip_initial:], chosen_vertices[skip_initial:]

# # Run (after solutions enabled):
# # start = (0.5, 0.3)
# # tri_points, tri_choices = play_chaos_game(triangle_vertices, start, num_steps=20000)
# # scatter_points(tri_points, title="Sierpiński Triangle (Chaos Game)", s=0.3)



### Task 5.4 — Experiment with the fraction
- Change the `fraction` from 0.5 to another value (e.g., 0.45 or 0.6).
- What happens to the pattern?


In [ ]:

# Try changing the fraction and re-plotting
# start = (0.5, 0.3)
# tri_points_045, _ = play_chaos_game(triangle_vertices, start, num_steps=20000, fraction=0.45)
# scatter_points(tri_points_045, title="Triangle with fraction=0.45", s=0.3)



---
<a id="carpet"></a>
## 6) Project 2: Sierpiński Carpet

Two approaches:
1) **Deterministic mask** on a \(3^n \times 3^n\) grid (remove center at each scale)  
2) **Chaos Game / IFS** with 8 maps (skip the center)



### Task 6.1 — Implement the carpet mask
Algorithm (base‑3 logic): A cell `(i,j)` is **removed** if at **any** digit place both `i` and `j` have digit `1` in base‑3.


In [ ]:
# Solution (uncomment):
# def sierpinski_carpet_mask(level):
#     n = 3 ** level
#     mask = np.ones((n, n), dtype=bool)
#     for i in range(n):
#         for j in range(n):
#             x, y = i, j
#             keep = True
#             for _ in range(level):
#                 if x % 3 == 1 and y % 3 == 1:
#                     keep = False; break
#                 x //= 3; y //= 3
#             mask[i, j] = keep
#     return mask

def show_mask(mask, title="Sierpiński Carpet (mask)"):
    plt.figure(figsize=(6,6))
    plt.imshow(mask, origin='lower', interpolation='nearest')
    plt.xticks([]); plt.yticks([])
    plt.title(title)
    plt.show()

# # After enabling solution:
# # for L in [1,2,3,4]:
# #     show_mask(sierpinski_carpet_mask(L), title=f"Carpet Mask (Level {L})")



### Task 6.2 — Chaos Game / IFS for the carpet
Use scale `1/3` and 8 offsets from the 3×3 grid **excluding the center**:  
`(0,0),(1,0),(2,0),(0,1),(2,1),(0,2),(1,2),(2,2)`



### (Optional) Timing comparison
Compare mask (level) vs. Chaos Game (number of points).


In [ ]:

def time_mask(level):
    t0 = time.time()
    m = sierpinski_carpet_mask(level)
    t1 = time.time()
    return (t1 - t0), m

def time_ifs(npts):
    t0 = time.time()
    pts = chaos_game_carpet(npts)
    t1 = time.time()
    return (t1 - t0), pts

# Example (after enabling solutions):
# tm, m = time_mask(5)   # 3**5 = 243
# ti, p = time_ifs(60000)
# print(f"Mask L=5: {tm:.3f}s, pixels={m.size}, filled={int(m.sum())}")
# print(f"IFS 60k pts: {ti:.3f}s, points={len(p)}")



---
<a id="reflect"></a>
## 7) Explore & Reflect

- How does changing the **fraction** affect the triangle pattern? Why?  
- For the carpet mask, derive the **filled ratio** after level `n` (hint: 8 of 9 squares remain each level).  
- Can you invent a new fractal by tweaking the offset set or using a different fraction?
